In [ ]:
!pip  install transformers


     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 3.4 MB/s 
     |████████████████████████████████| 895 kB 36.8 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.8 MB 35.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW

In [ ]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

In [ ]:
!pip install emoji

     |████████████████████████████████| 174 kB 4.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=bd61e40356d8bcdf0d36c35bb9b5d078a897a12e8842584d7e659123a44d476c
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class Bert_Tweet_Extra_dropout_Model_Arch(nn.Module):
    def __init__(self,pre_trained='vinai/bertweet-base'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained, output_hidden_states=True)
        output_channel = 16  # number of kernels
        num_classes = 6  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        ks = 3  # three conv nets here

        # input_channel = word embeddings at a value of 1; 3 for RGB images
        input_channel = 4  # for single embedding, input_channel = 1

        # [3, 4, 5] = window height
        # padding = padding to account for height of search window

        # 3 convolutional nets
        self.conv1 = nn.Conv2d(input_channel, output_channel, (3, embedding_dim), padding=(2, 0), groups=4)
        self.conv2 = nn.Conv2d(input_channel, output_channel, (4, embedding_dim), padding=(3, 0), groups=4)
        self.conv3 = nn.Conv2d(input_channel, output_channel, (5, embedding_dim), padding=(4, 0), groups=4)

        # apply dropout
        self.dropout = nn.Dropout(dropout)

        # fully connected layer for classification
        # 3x conv nets * output channel
        self.fc1 = nn.Linear(ks * output_channel, num_classes)
        self.softmax = nn.Sigmoid()

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.bert(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        hidden_layers = outputs[2]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        x = hidden_layers[:, -4:] 
        # x = x.unsqueeze(1)
        # x = torch.mean(x, 0)
        # print(hidden_layers.size())
      
        torch.cuda.empty_cache()
        x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        x = [F.dropout(i,0.65) for i in x]
        # max-over-time pooling; # (batch, channel_output) * ks
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # concat results; (batch, channel_output * ks)
        x = torch.cat(x, 1)
        # add dropout
        x = self.dropout(x)
        # generate logits (batch, target_size)
        logit = self.fc1(x)
        torch.cuda.empty_cache()
        return logit

In [ ]:
class Bert_tweet_Model_Arch(nn.Module):
    def __init__(self,pre_trained='vinai/bertweet-base'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained, output_hidden_states=True)
        output_channel = 16  # number of kernels
        num_classes = 6  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        ks = 3  # three conv nets here

        # input_channel = word embeddings at a value of 1; 3 for RGB images
        input_channel = 4  # for single embedding, input_channel = 1

        # [3, 4, 5] = window height
        # padding = padding to account for height of search window

        # 3 convolutional nets
        self.conv1 = nn.Conv2d(input_channel, output_channel, (3, embedding_dim), padding=(2, 0), groups=4)
        self.conv2 = nn.Conv2d(input_channel, output_channel, (4, embedding_dim), padding=(3, 0), groups=4)
        self.conv3 = nn.Conv2d(input_channel, output_channel, (5, embedding_dim), padding=(4, 0), groups=4)

        # apply dropout
        self.dropout = nn.Dropout(dropout)

        # fully connected layer for classification
        # 3x conv nets * output channel
        self.fc1 = nn.Linear(ks * output_channel, num_classes)
        self.softmax = nn.Sigmoid()

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.bert(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        hidden_layers = outputs[2]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        x = hidden_layers[:, -4:] 
        # x = x.unsqueeze(1)
        # x = torch.mean(x, 0)
        # print(hidden_layers.size())
      
        torch.cuda.empty_cache()
        x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        # max-over-time pooling; # (batch, channel_output) * ks
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # concat results; (batch, channel_output * ks)
        x = torch.cat(x, 1)
        # add dropout
        x = self.dropout(x)
        # generate logits (batch, target_size)
        logit = self.fc1(x)
        torch.cuda.empty_cache()
        return logit

In [ ]:
class Roberta_Model_Arch(nn.Module):
    def __init__(self,pre_trained='roberta-base'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained, output_hidden_states=True)
        output_channel = 16  # number of kernels
        num_classes = 6  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        ks = 3  # three conv nets here

        # input_channel = word embeddings at a value of 1; 3 for RGB images
        input_channel = 4  # for single embedding, input_channel = 1

        # [3, 4, 5] = window height
        # padding = padding to account for height of search window

        # 3 convolutional nets
        self.conv1 = nn.Conv2d(input_channel, output_channel, (3, embedding_dim), padding=(2, 0), groups=4)
        self.conv2 = nn.Conv2d(input_channel, output_channel, (4, embedding_dim), padding=(3, 0), groups=4)
        self.conv3 = nn.Conv2d(input_channel, output_channel, (5, embedding_dim), padding=(4, 0), groups=4)

        # apply dropout
        self.dropout = nn.Dropout(dropout)

        # fully connected layer for classification
        # 3x conv nets * output channel
        self.fc1 = nn.Linear(ks * output_channel, num_classes)
        self.softmax = nn.Sigmoid()

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.bert(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        hidden_layers = outputs[2]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        x = hidden_layers[:, -7:-3] 
        # x = x.unsqueeze(1)
        # x = torch.mean(x, 0)
        # print(hidden_layers.size())
      
        torch.cuda.empty_cache()
        x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        x = [F.dropout(i) for i in x]
        # max-over-time pooling; # (batch, channel_output) * ks
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # concat results; (batch, channel_output * ks)
        x = torch.cat(x, 1)
        # add dropout
        x = self.dropout(x)
        # generate logits (batch, target_size)
        logit = self.fc1(x)
        torch.cuda.empty_cache()
        return logit

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['text_ids'].to(device, dtype = torch.long)
        mask = data['text_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        sig=nn.Sigmoid()
        outputs=sig(outputs)
        # outputs = outputs.argmax(dim=1)
#         print(len(outputs))
#         print(len(np.max(outputs.cpu().detach().numpy(),axis=1)))
        PREDS.append(outputs.detach().cpu().numpy()) 
        # print(outputs.detach().cpu().numpy())
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device,model):
    final_preds = []
    for i, path in enumerate(model_paths):
        model.to(device)
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    # print(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    # print(final_preds)
    final_preds[final_preds>=0.5] = 1
    final_preds[final_preds<0.5] = 0
    # final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [ ]:
class PCLTrainDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
       
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # summary = self.summary[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
                            
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
        
       
        
        
        return {
            
            'text_ids': torch.tensor(text_ids, dtype=torch.long),
            'text_mask': torch.tensor(text_mask, dtype=torch.long),
            
        }


In [ ]:
tokenizer_bert_tweet= AutoTokenizer.from_pretrained('vinai/bertweet-base')
tokenizer_roberta= AutoTokenizer.from_pretrained('roberta-base')


Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/824k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/ISarcasm/Test_dataset/taskA.En.input.csv')

In [ ]:
valid_dataset = PCLTrainDataset(test, tokenizer=tokenizer_bert_tweet, max_length=128)
valid_loader = DataLoader(valid_dataset, batch_size=32, 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_asymm_extra_dropout_middle_layers/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_asymm_extra_dropout_middle_layers/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_asymm_extra_dropout_middle_layers/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_asymm_extra_dropout_middle_layers/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_asymm_extra_dropout_middle_layers/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_bert_tweet_extra_dropout = inference(MODEL_PATH_2, valid_loader, 'cuda',Bert_Tweet_Extra_dropout_Model_Arch())

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 44/44 [00:26<00:00,  1.66it/s]


Getting predictions for model 2


...

100%|██████████| 44/44 [00:26<00:00,  1.63it/s]


Getting predictions for model 4


100%|██████████| 44/44 [00:26<00:00,  1.63it/s]


Getting predictions for model 5


100%|██████████| 44/44 [00:27<00:00,  1.62it/s]


In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_balanced_loss_random_sampler/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_balanced_loss_random_sampler/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_balanced_loss_random_sampler/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_balanced_loss_random_sampler/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_balanced_loss_random_sampler/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_bert_tweet = inference(MODEL_PATH_2, valid_loader, 'cuda',Bert_tweet_Model_Arch())

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 44/44 [00:26<00:00,  1.63it/s]


Getting predictions for model 2


100%|██████████| 44/44 [00:26<00:00,  1.64it/s]


...

100%|██████████| 44/44 [00:26<00:00,  1.63it/s]


Getting predictions for model 4


100%|██████████| 44/44 [00:26<00:00,  1.65it/s]


Getting predictions for model 5


100%|██████████| 44/44 [00:26<00:00,  1.64it/s]


In [ ]:
valid_dataset = PCLTrainDataset(test, tokenizer=tokenizer_roberta, max_length=128)
valid_loader = DataLoader(valid_dataset, batch_size=32, 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models_Task_B/roberta_asymm__7_3_extra_dropout/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_roberta = inference(MODEL_PATH_2, valid_loader, 'cuda',Roberta_Model_Arch())

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 44/44 [00:26<00:00,  1.69it/s]


Getting predictions for model 2


100%|██████████| 44/44 [00:26<00:00,  1.67it/s]


...

100%|██████████| 44/44 [00:26<00:00,  1.68it/s]


Getting predictions for model 4


100%|██████████| 44/44 [00:26<00:00,  1.67it/s]


Getting predictions for model 5


100%|██████████| 44/44 [00:26<00:00,  1.67it/s]


In [ ]:
df_bert_tweet_extra_dropout = pd.DataFrame()
df_bert_tweet_extra_dropout['pred_0']=preds_bert_tweet_extra_dropout[:,0]
df_bert_tweet_extra_dropout['pred_1']=preds_bert_tweet_extra_dropout[:,1]
df_bert_tweet_extra_dropout['pred_2']=preds_bert_tweet_extra_dropout[:,2]
df_bert_tweet_extra_dropout['pred_3']=preds_bert_tweet_extra_dropout[:,3]
df_bert_tweet_extra_dropout['pred_4']=preds_bert_tweet_extra_dropout[:,4]
df_bert_tweet_extra_dropout['pred_5']=preds_bert_tweet_extra_dropout[:,5]

In [ ]:
df_bert_tweet = pd.DataFrame()
df_bert_tweet['pred_0']=preds_bert_tweet[:,0]
df_bert_tweet['pred_1']=preds_bert_tweet[:,1]
df_bert_tweet['pred_2']=preds_bert_tweet[:,2]
df_bert_tweet['pred_3']=preds_bert_tweet[:,3]
df_bert_tweet['pred_4']=preds_bert_tweet[:,4]
df_bert_tweet['pred_5']=preds_bert_tweet[:,5]

In [ ]:
df_roberta = pd.DataFrame()
df_roberta['pred_0']=preds_roberta[:,0]
df_roberta['pred_1']=preds_roberta[:,1]
df_roberta['pred_2']=preds_roberta[:,2]
df_roberta['pred_3']=preds_roberta[:,3]
df_roberta['pred_4']=preds_roberta[:,4]
df_roberta['pred_5']=preds_roberta[:,5]

In [ ]:
df_svm=pd.read_csv('/content/drive/MyDrive/ISarcasm/ML_task_B_models/svm_model_prediction_ML_FE_4.csv')
df_svm_bagging=pd.read_csv('/content/drive/MyDrive/ISarcasm/ML_task_B_models/svm_model_prediction_ML_FE_3.csv')

In [ ]:
df_pred_final=pd.DataFrame()

In [ ]:
all_data = [df_roberta['pred_0'] , 
             df_bert_tweet['pred_0'],
            df_bert_tweet_extra_dropout['pred_0'], 
            df_svm['pred_0'], 
            df_svm_bagging['pred_0'], 
           
              ]
votes  = pd.concat(all_data, axis='columns')
predictions = votes.mode(axis='columns').to_numpy()
df_pred_final['label_1']=predictions.flatten()


In [ ]:
votes

,pred_0,pred_0,pred_0,pred_0,pred_0
0,1.0,1.0,1.0,0,1
1,1.0,0.0,1.0,1,1
2,1.0,1.0,1.0,0,1
3,1.0,1.0,1.0,1,0
4,1.0,1.0,1.0,0,1
...,...,...,...,...,...
1395,1.0,1.0,1.0,0,0
1396,1.0,1.0,1.0,0,0
1397,1.0,1.0,1.0,1,1
1398,1.0,1.0,1.0,0,1


In [ ]:
all_data = [df_roberta['pred_1'] , 
             df_bert_tweet['pred_1'],
            df_bert_tweet_extra_dropout['pred_1'], 
            df_svm['pred_1'], 
            df_svm_bagging['pred_1'], 
           
              ]
votes  = pd.concat(all_data, axis='columns')
predictions = votes.mode(axis='columns').to_numpy()
df_pred_final['label_2']=predictions.flatten()


In [ ]:
votes

,pred_1,pred_1,pred_1,pred_1,pred_1
0,1.0,0.0,0.0,1,0
1,1.0,1.0,1.0,0,0
2,1.0,0.0,0.0,1,0
3,0.0,0.0,0.0,0,1
4,0.0,0.0,0.0,1,0
...,...,...,...,...,...
1395,1.0,0.0,0.0,1,1
1396,1.0,0.0,0.0,1,1
1397,1.0,0.0,0.0,0,0
1398,1.0,0.0,1.0,1,0


In [ ]:
all_data = [df_roberta['pred_2'] , 
             df_bert_tweet['pred_2'],
            df_bert_tweet_extra_dropout['pred_2'], 
            df_svm['pred_2'], 
            df_svm_bagging['pred_2'], 
           
              ]
votes  = pd.concat(all_data, axis='columns')
predictions = votes.mode(axis='columns').to_numpy()
df_pred_final['label_3']=predictions.flatten()


In [ ]:
all_data = [df_roberta['pred_3'] , 
             df_bert_tweet['pred_3'],
            df_bert_tweet_extra_dropout['pred_3'], 
            df_svm['pred_3'], 
            df_svm_bagging['pred_3'], 
           
              ]
votes  = pd.concat(all_data, axis='columns')
predictions = votes.mode(axis='columns').to_numpy()
df_pred_final['label_4']=predictions.flatten()


In [ ]:
votes

,pred_3,pred_3,pred_3,pred_3,pred_3
0,0.0,0.0,1.0,1,0
1,0.0,0.0,0.0,1,0
2,0.0,0.0,0.0,0,0
3,0.0,0.0,0.0,0,0
4,0.0,0.0,0.0,1,0
...,...,...,...,...,...
1395,0.0,0.0,0.0,0,0
1396,0.0,0.0,0.0,1,0
1397,0.0,0.0,0.0,0,0
1398,0.0,0.0,0.0,0,0


In [ ]:
all_data = [df_roberta['pred_4'] , 
             df_bert_tweet['pred_4'],
            df_bert_tweet_extra_dropout['pred_4'], 
            df_svm['pred_4'], 
            df_svm_bagging['pred_4'], 
           
              ]
votes  = pd.concat(all_data, axis='columns')
predictions = votes.mode(axis='columns').to_numpy()
df_pred_final['label_5']=predictions.flatten()


In [ ]:
all_data = [df_roberta['pred_5'] , 
             df_bert_tweet['pred_5'],
            df_bert_tweet_extra_dropout['pred_5'], 
            df_svm['pred_5'], 
            df_svm_bagging['pred_5'], 
           
              ]
votes  = pd.concat(all_data, axis='columns')
predictions = votes.mode(axis='columns').to_numpy()
df_pred_final['label_6']=predictions.flatten()


In [ ]:
df_pred_final['label_5'].unique()

array([0., 1.])

In [ ]:
df_pred_final

,label_1,label_2,label_3,label_4,label_5,label_6
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1395,1.0,1.0,0.0,0.0,0.0,0.0
1396,1.0,1.0,0.0,0.0,0.0,1.0
1397,1.0,0.0,0.0,0.0,1.0,0.0
1398,1.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
predictions=df_pred_final[['label_1','label_2','label_3','label_4','label_5','label_6']].values

In [ ]:
predictions.flatten()

array([1., 0., 0., ..., 0., 0., 0.])

In [ ]:
predictions[0]

array([1., 0., 0., 0., 0., 0.])

In [ ]:
predictions[1]

array([1., 1., 0., 0., 0., 0.])

In [ ]:
for pred in predictions:
  print(pred)
  break

[1. 0. 0. 0. 0. 0.]


In [ ]:
with open('/content/drive/MyDrive/ISarcasm/Test_prediction/task_b.txt', 'w') as f:
    print('sarcasm,irony,satire,understatement,overstatement,rhetorical_question', file=f)
    for i in range(0,len(predictions)):
      # print(pred)
      print(int(predictions[i][0]),',',int(predictions[i][1]),',',int(predictions[i][2]),',',int(predictions[i][3]),',',int(predictions[i][4]),',',int(predictions[i][5]), file=f)

In [ ]:
prediction_final

In [ ]:
prediction_list